In [1]:
# import libraries
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Function to create a coordinate frame
def create_coordinate_frame(size=0.2, origin=[0, 0, 0]):
    return o3d.geometry.TriangleMesh.create_coordinate_frame(size=size, origin=origin)

# Function to reorient the point cloud
def reorient_point_cloud(pcd):
    # Estimate the plane using RANSAC
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                             ransac_n=3,
                                             num_iterations=1000)
    [a, b, c, d] = plane_model

    # Compute the normal vector to the plane
    normal_vector = np.array([a, b, c])

    # Compute the rotation axis (cross product with x-axis)
    x_axis = np.array([1, 0, 0])
    rotation_axis = np.cross(normal_vector, x_axis)
    rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)

    # Compute the angle between the normal vector and x-axis
    theta = np.arccos(np.dot(normal_vector, x_axis) / np.linalg.norm(normal_vector))

    # Compute the rotation matrix using Rodrigues' rotation formula
    K = np.array([[0, -rotation_axis[2], rotation_axis[1]],
                  [rotation_axis[2], 0, -rotation_axis[0]],
                  [-rotation_axis[1], rotation_axis[0], 0]])

    R = np.eye(3) + np.sin(theta) * K + (1 - np.cos(theta)) * np.dot(K, K)

    # Rotate the point cloud
    pcd.rotate(R, center=(0, 0, 0))

    # Find the centroid of the floor points
    floor_points = pcd.select_by_index(inliers)
    centroid = floor_points.get_center()

    # Translate the point cloud to move the centroid to the origin
    translation = -centroid
    pcd.translate(translation)

    return pcd

# Function to apply random transformations to a point cloud
def apply_random_transformations(pcd):
    # Apply random rotation
    R = pcd.get_rotation_matrix_from_xyz(np.random.uniform(0, 2*np.pi, 3))
    pcd.rotate(R, center=(0, 0, 0))

    # Apply random translation
    translation = np.random.uniform(-5, 5, 3)
    pcd.translate(translation)

    return pcd

# Unit test case
def test_reorientation(input_pcd_file):
    # Load the input point cloud
    original_pcd = o3d.io.read_point_cloud(input_pcd_file)
    o3d.visualization.draw_geometries([original_pcd, create_coordinate_frame()], window_name="Original Point Cloud")

    # Apply random transformations
    transformed_pcd = apply_random_transformations(original_pcd)
    o3d.visualization.draw_geometries([transformed_pcd, create_coordinate_frame()], window_name="Transformed Point Cloud")

    # Reorient the transformed point cloud
    reoriented_pcd = reorient_point_cloud(transformed_pcd)
    o3d.visualization.draw_geometries([reoriented_pcd, create_coordinate_frame()], window_name="Reoriented Point Cloud")

    # Check if the floor is correctly aligned
    plane_model, inliers = reoriented_pcd.segment_plane(distance_threshold=0.01,
                                                        ransac_n=3,
                                                        num_iterations=1000)
    [a, b, c, d] = plane_model

    # The floor should now be on the YZ plane, hence a should be close to 1 and b, c should be close to 0
    assert np.isclose(a, 1, atol=0.1), f"Expected a to be close to 1, got {a}"
    assert np.isclose(b, 0, atol=0.1), f"Expected b to be close to 0, got {b}"
    assert np.isclose(c, 0, atol=0.1), f"Expected c to be close to 0, got {c}"

    # The centroid of the floor should be at the origin
    floor_points = reoriented_pcd.select_by_index(inliers)
    centroid = floor_points.get_center()
    assert np.allclose(centroid, [0, 0, 0], atol=0.1), f"Expected centroid to be close to [0, 0, 0], got {centroid}"

    print("Test passed!")


In [3]:

# Run the unit test with your input point cloud file
test_reorientation("D:\Rohit\JObs\AvataarAI\pc_assignment\shoe_pc.ply")


Test passed!
